<a href="https://colab.research.google.com/github/the-faisalahmed/Optimization/blob/main/Lying_about_your_Age.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lying about your Age

![](https://deliterate.wordpress.com/wp-content/uploads/2013/11/58452819_adventures_of_pinocchio_answer_1_xlarge.jpeg?w=144) ![](https://deliterate.wordpress.com/wp-content/uploads/2013/11/58452819_adventures_of_pinocchio_answer_1_xlarge.jpeg?w=144) ![](https://deliterate.wordpress.com/wp-content/uploads/2013/11/58452819_adventures_of_pinocchio_answer_1_xlarge.jpeg?w=144) ![](https://deliterate.wordpress.com/wp-content/uploads/2013/11/58452819_adventures_of_pinocchio_answer_1_xlarge.jpeg?w=144) ![](https://deliterate.wordpress.com/wp-content/uploads/2013/11/58452819_adventures_of_pinocchio_answer_1_xlarge.jpeg?w=144)

Annie, Betty, Carrie, Darla, and Eve recently found out that all of their birthdays were on the same day, though they are different ages. On their mutual birthday, they were jabbering away, flapping their gums about their recent discovery. And, lucky me, I was there. Some of the things that I overheard were...

1. Darla said to Betty: "I'm nine years older than Eve."
2. Eve said to Betty: "I'm seven years older than Annie."
3. Annie said to Betty: "Your age is exactly 70% greater than mine."
4.  Betty said to Carrie: "Eve is younger than you."
5.  Carrie said to Darla: "The difference between our ages is six years."
6.  Carrie said to Annie: "I'm ten years older than you."
7.  Carrie said to Annie: "Betty is younger than Darla."
8. Betty said to Carrie: "The difference between your age and Darla's is the same as the difference between Darla's and Eve's."

Since I knew these people -- and how old they were, I knew that they were not telling the whole truth. After thinking about it, I realized that when one of them spoke to somebody older than herself, everything she said was true, but when speaking to somebody younger, everything she said was false.

How old is each person?

In [1]:
!pip install ortools
from ortools.sat.python import cp_model

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.27.2 which is incompatible.
google-ai-generativelanguage 0.6.4 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.2 which is incompatible.
google-api-core 2.16.2 requires protobuf!=3.20.0,

In [21]:
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, x,y):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__x = x
        self.__y = y
        self.__solution_count = 0


    def on_solution_callback(self):
        self.__solution_count += 1

        print(f'Solution {self.__solution_count}')
        print([(i,self.Value(self.__x[i])) for i in self.__x])
        print([(i,self.Value(self.__y[i])) for i in self.__y])
        print('\n')

    def solution_count(self):
        return self.__solution_count

In [22]:
def SearchForAllSolutionsSampleSat():
  # Creates the model and set solver
  model = cp_model.CpModel()
  solver = cp_model.CpSolver()

  max_age = 100
  num_constraints = 8

  people = ['Annie', 'Betty', 'Carrie', 'Darla', 'Eve' ]

  x = {i:model.NewIntVar(1,max_age,f"x_{i}") for i in people}

  y = {i:model.NewBoolVar(f"y_{i}") for i in range(1,num_constraints+1)}

  # All different ages
  model.AddAllDifferent([x[i] for i in x.keys()])

  # 1. Darla said to Betty: "I'm nine years older than Eve."
  model.Add(x['Darla'] < x['Betty']).OnlyEnforceIf(y[1])
  model.Add(x['Darla'] >= x['Betty']).OnlyEnforceIf(~y[1])
  model.Add(x['Darla'] == x['Eve'] + 9).OnlyEnforceIf(y[1])

  # 2. Eve said to Betty: "I'm seven years older than Annie."
  model.Add(x['Eve'] < x['Betty']).OnlyEnforceIf(y[2])
  model.Add(x['Eve'] >= x['Betty']).OnlyEnforceIf(~y[2])
  model.Add(x['Eve'] == x['Annie'] + 7).OnlyEnforceIf(y[2])

  # 3. Annie said to Betty: "Your age is exactly 70% greater than mine."
  model.Add(x['Annie'] < x['Betty']).OnlyEnforceIf(y[3])
  model.Add(x['Annie'] >= x['Betty']).OnlyEnforceIf(~y[3])
  model.Add(170*x['Betty'] == 10*x['Annie']).OnlyEnforceIf(y[3])

  # 4.  Betty said to Carrie: "Eve is younger than you."
  model.Add(x['Betty'] < x['Carrie']).OnlyEnforceIf(y[4])
  model.Add(x['Betty'] >= x['Carrie']).OnlyEnforceIf(~y[4])
  model.Add(x['Eve'] < x['Carrie']).OnlyEnforceIf(y[4])

  # 5.  Carrie said to Darla: "The difference between our ages is six years."
  model.Add(x['Carrie'] < x['Darla']).OnlyEnforceIf(y[5])
  model.Add(x['Carrie'] >= x['Darla']).OnlyEnforceIf(~y[5])
  model.Add(x['Darla'] - x['Carrie'] == 6).OnlyEnforceIf(y[5])

  # 6.  Carrie said to Annie: "I'm ten years older than you."
  model.Add(x['Carrie'] < x['Annie']).OnlyEnforceIf(y[6])
  model.Add(x['Carrie'] >= x['Annie']).OnlyEnforceIf(~y[6])
  model.Add(x['Carrie'] == x['Annie'] + 10).OnlyEnforceIf(y[6])

  # 7.  Carrie said to Annie: "Betty is younger than Darla."
  model.Add(x['Carrie'] < x['Annie']).OnlyEnforceIf(y[7])
  model.Add(x['Carrie'] >= x['Annie']).OnlyEnforceIf(~y[7])
  model.Add(x['Betty'] < x['Darla']).OnlyEnforceIf(y[7])

  # 8. Betty said to Carrie: "The difference between your age and Darla's is the same as the difference between Darla's and Eve's."
  model.Add(x['Betty'] < x['Carrie']).OnlyEnforceIf(y[8])
  model.Add(x['Betty'] >= x['Carrie']).OnlyEnforceIf(~y[8])
  model.Add(x['Carrie'] - x['Darla'] == x['Darla'] - x['Eve']).OnlyEnforceIf(y[8])

  # Enumerate all solutions.
  solver.parameters.enumerate_all_solutions = False

  solution_printer = VarArraySolutionPrinter(x,y)

  status = solver.Solve(model, solution_printer)

  print(f"Status = {solver.StatusName(status)}")

SearchForAllSolutionsSampleSat()

Solution 1
[('Annie', 2), ('Betty', 1), ('Carrie', 5), ('Darla', 4), ('Eve', 3)]
[(1, 0), (2, 0), (3, 0), (4, 1), (5, 0), (6, 0), (7, 0), (8, 1)]


Solution 2
[('Annie', 97), ('Betty', 1), ('Carrie', 98), ('Darla', 50), ('Eve', 2)]
[(1, 0), (2, 0), (3, 0), (4, 1), (5, 0), (6, 0), (7, 0), (8, 1)]


Status = OPTIMAL
